# Data exploration (task 1)

## Load data

In [1]:
# Start dask cluster
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

2023-03-23 12:31:05,984 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-n9pmxnhu', purging
2023-03-23 12:31:05,984 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wcduebuf', purging
2023-03-23 12:31:05,985 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-w5fiikvn', purging
2023-03-23 12:31:05,985 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-xs6_77d_', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41875,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.45 GiB
Comm: tcp://127.0.0.1:41591,Total threads: 2
Dashboard: http://127.0.0.1:43841/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:33825,


In [2]:
# Connect to my servers S3 bucket
import dask.dataframe as dd
df = dd.read_parquet(
    "s3://fakenews/shuffled_deduped/shuffled_deduped.parquet",
    storage_options={
        'anon': False,
        'key': 'kristian',
        'secret': 'kristianersej',
        'client_kwargs': {
            'endpoint_url': 'http://172.25.133.208:9000'
        }
    }
)
df.head()

,id,type,content,title
0,4171095,fake,New Catholics remind us to take faith seriousl...,New Catholics remind us to take faith seriousl...
1,1140817,rumor,The Spaniard has endured a frustrating couple ...,Chelsea can't win the title without this playe...
2,2617658,political,"How Sony, Obama, Seth Rogen and the CIA Secret...","How Sony, Obama, Seth Rogen and the CIA Secret..."
3,7261250,junksci,by PAUL FASSA “If the doors of perception wer...,A 65 Year Old Woman with Cancer Trips Out on S...
4,8078086,reliable,Follow @ZeeNews His long-winded political ran...,"Fidel Castro: Revolutionary, renaissance man |..."


## Summary statistics

In [4]:
df.type.value_counts().compute()

reliable      1808242
political      972306
bias           771419
fake           770341
conspiracy     495436
rumor          376815
unknown        231320
clickbait      176403
unreliable     104651
satire          85526
junksci         79435
hate            65265
Name: type, dtype: int64

In [5]:
# Split into train and test
from dask_ml.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.head()

/home/kristiandampedersen/.local/share/virtualenvs/fakenewsproject-Oj_5wi4E/lib/python3.10/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


,id,type,content,title
0,4171095,fake,New Catholics remind us to take faith seriousl...,New Catholics remind us to take faith seriousl...
1,1140817,rumor,The Spaniard has endured a frustrating couple ...,Chelsea can't win the title without this playe...
2,2617658,political,"How Sony, Obama, Seth Rogen and the CIA Secret...","How Sony, Obama, Seth Rogen and the CIA Secret..."
3,7261250,junksci,by PAUL FASSA “If the doors of perception wer...,A 65 Year Old Woman with Cancer Trips Out on S...
6,5568066,fake,Volatility and Precious Metals (Before It's N...,Volatility and Precious Metals


In [6]:
# Write test to s3 for later user
df_test.to_parquet(
    "s3://fakenews/shuffled_deduped/test.parquet",
    storage_options={
        'anon': False,
        'key': 'kristian',
        'secret': 'kristianersej',
        'client_kwargs': {
            'endpoint_url': 'http://172.25.133.208:9000'
        }
    }
)

KeyboardInterrupt: 

In [14]:
# Remove test.parquet from s3
import s3fs
fs = s3fs.S3FileSystem(
    anon=False,
    key='kristian',
    secret='kristianersej',
    client_kwargs={
        'endpoint_url': 'http://172.25.133.208:9000'
    }
)
fs.rm("fakenews/shuffled_deduped/test.parquet/*")

In [16]:
fs.ls("fakenews/shuffled_deduped/")

['fakenews/shuffled_deduped/',
 'fakenews/shuffled_deduped/shuffled_deduped.parquet']